In [26]:
import numpy as np

from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

from collections import defaultdict

def fixed_xor(b1, b2):
    a = np.frombuffer(b1, 'u1')
    b = np.frombuffer(b2, 'u1')
    return (a ^ b).tobytes()

def cbc(plain, key, iv):

    aes = Cipher(algorithms.AES128(key), modes.ECB()).encryptor()

    cipher = b''
    cipher_blk = iv

    plain_blks = np.frombuffer(plain, 'u1').reshape(-1, 16)

    for blk in plain_blks:
        cipher_blk = aes.update(fixed_xor(blk.tobytes(), cipher_blk))
        cipher += cipher_blk
    
    return cipher

def ecb(plain, key):
    aes = Cipher(algorithms.AES128(key), modes.ECB()).encryptor()
    return aes.update(plain)

def pkcs7(b, size=128):
    bsz = len(b)
    size //= 8
    sz = size * (bsz // size + 1)
    return b + bytes([sz - bsz]) * (sz - bsz)

In [2]:
rand_key = lambda: np.random.bytes(16)

In [45]:
def encryption_oracle(plain):
    """Emulates an unkown encryption oracle that might either use ECB or CBC modes"""

    key = rand_key()

    prefix = np.random.bytes(np.random.randint(5, 11))
    suffix = np.random.bytes(np.random.randint(5, 11))

    plain = pkcs7(prefix + plain + suffix)
    plain_blks = np.frombuffer(plain, 'u1').reshape(-1, 16)

    cipher = b''

    for blk in plain_blks:
        if np.random.randint(1, 3) % 2:
            cipher += ecb(blk.tobytes(), key)
        else:
            cipher += cbc(blk.tobytes(), key, rand_key())
    
    return cipher

In [40]:
def detect(cipher):
    counts = defaultdict(int)

    for blk in np.frombuffer(cipher, 'u1').reshape(-1, 16):
        counts[blk.tobytes()] += 1
    
    return 'ecb' if np.any(np.array(list(counts.values())) > 1) else 'cbc'

In [44]:
detect(encryption_oracle(b'A' * (20+16*2)))

'ecb'